In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-4.9 g++-4.9 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

65 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2020-04-20 11:21:11--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?eGrOE7aznKvvgSk_0qAhgJSYkMwXER28HTIggp1ZSGrf1hbneXIS3CtSIIYW8nWZs7fU7UHfA-E_ttxZLY4Ykp3zkT4CJ055UednCPv7zKnE-PYM_FPKMFJUgoqMMfuJcf9_HB01NWbYh4Q8NiPZ4AEoQ-u9DtkoIIKDLD-6o6bGHPnwvInxVlfNrm4nQxtE0wcPms7TnUew9pG3Z6sZXfqa0w [following]
--2020-04-20 11:21:12--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?eGrOE7aznKvvgSk_0qAhgJSYkMwX

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-k37oyn6a
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-k37oyn6a
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=4cec03c3ea7b9ed234b22f7ad4dfc34c30cfb8de5d97e7b37bc5393e7f8d968f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lhqzaxwt/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu
#include "utils.h"
#include<device_launch_parameters.h>
#include<device_functions.h>

__global__ void shmem_reduce_kernel(float * d_out, const float * const d_in, bool is_max)
{
  // sdata is allocated in the kernel call: 3rd arg to <<<b, t, shmem>>>
  extern __shared__ float sdata[];

  int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int tid = threadIdx.x;

  // load shared mem from global mem
  sdata[tid] = d_in[myId];
  __syncthreads();            // make sure entire block is loaded!

  // do reduction in shared mem
  for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1)
  {
    if (tid < s)
    {
      if (is_max)
        sdata[tid] = max(sdata[tid], sdata[tid + s]);
      else
        sdata[tid] = min(sdata[tid], sdata[tid + s]);
    }
    __syncthreads();        // make sure all adds at one stage are done!
  }

  // only thread 0 writes result for this block back to global mem
  if (tid == 0)
  {
    d_out[blockIdx.x] = sdata[0];
  }
}

__global__ void histo_kernel(unsigned int * d_out, const float * const d_in,
  const size_t numBins, float logLumRange, float min_logLum)
{
  int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int bin = (d_in[myId] - min_logLum) / logLumRange * numBins;
  if (bin == numBins)  bin--;
  atomicAdd(&d_out[bin], 1);
}

__global__ void scan_kernel(unsigned int * d_out, const float * const d_in,
  const size_t numBins, float logLumRange, float min_logLum)
{
  int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int bin = (d_in[myId] - min_logLum) / logLumRange * numBins;
  if (bin == numBins)  bin--;
  atomicAdd(&d_out[bin], 1);
}

// Hillis Steele Scan - described in lecture
__global__ void cdf_kernel(unsigned int * d_in, const size_t numBins)
{
  int myId = threadIdx.x;
  for (int d = 1; d < numBins; d *= 2) {
    if ((myId + 1) % (d * 2) == 0) {
      d_in[myId] += d_in[myId - d];
    }
    __syncthreads();
  }
  if (myId == numBins - 1) d_in[myId] = 0;
  for (int d = numBins / 2; d >= 1; d /= 2) {
    if ((myId + 1) % (d * 2) == 0) {
      unsigned int tmp = d_in[myId - d];
      d_in[myId - d] = d_in[myId];
      d_in[myId] += tmp;
    }
    __syncthreads();
  }
}

// Blelloch Scan - described in lecture
__global__ void cdf_kernel_2(unsigned int * d_in, const size_t numBins)
{ 
  int idx = threadIdx.x;
  extern __shared__ int temp[];
  int pout = 0, pin = 1;

  temp[idx] = (idx > 0) ? d_in[idx - 1] : 0;
  __syncthreads();

  for (int offset = 1; offset < n; offset *= 2) {
    // swap double buffer indices
    pout = 1 - pout;
    pin = 1 - pout;
    if (idx >= offset) {
      temp[pout*n+idx] = temp[pin*n+idx - offset] + temp[pin*n+idx];  // changed line
    } else {
      temp[pout*n+idx] = temp[pin*n+idx];
    }
    __syncthreads();
  }
  d_in[idx] = temp[pout*n+idx];
}

void your_histogram_and_prefixsum(const float* const d_logLuminance,
  unsigned int* const d_cdf,
  float &min_logLum,
  float &max_logLum,
  const size_t numRows,
  const size_t numCols,
  const size_t numBins)
{
  //TODO
  /*Here are the steps you need to implement
  1) find the minimum and maximum value in the input logLuminance channel
  store in min_logLum and max_logLum */
  // use reduce
  const int m = 1 << 10;
  int blocks = ceil((float)numCols * numRows / m);

  float *d_intermediate; // should not modify d_in
  checkCudaErrors(cudaMalloc(&d_intermediate, sizeof(float)* blocks)); // store max and min
  float *d_min, *d_max;
  checkCudaErrors(cudaMalloc((void **)&d_min, sizeof(float)));
  checkCudaErrors(cudaMalloc((void **)&d_max, sizeof(float)));

  shmem_reduce_kernel << <blocks, m, m * sizeof(float) >> >(d_intermediate, d_logLuminance, true);
  shmem_reduce_kernel << <1, blocks, blocks * sizeof(float) >> >(d_max, d_intermediate, true);
  shmem_reduce_kernel << <blocks, m, m * sizeof(float) >> >(d_intermediate, d_logLuminance, false);
  shmem_reduce_kernel << <1, blocks, blocks * sizeof(float) >> >(d_min, d_intermediate, false);
  checkCudaErrors(cudaMemcpy(&min_logLum, d_min, sizeof(float), cudaMemcpyDeviceToHost));
  checkCudaErrors(cudaMemcpy(&max_logLum, d_max, sizeof(float), cudaMemcpyDeviceToHost));

  checkCudaErrors(cudaFree(d_intermediate));
  checkCudaErrors(cudaFree(d_min));
  checkCudaErrors(cudaFree(d_max));

  //2) subtract them to find the range
  float logLumRange = max_logLum - min_logLum;
  printf("max_logLum: %f  min_logLum: %f  logLumRange: %f\n", max_logLum, min_logLum, logLumRange);

  //3) generate a histogram of all the values in the logLuminance channel using
  //   the formula: bin = (lum[i] - lumMin) / lumRange * numBins
  checkCudaErrors(cudaMemset(d_cdf, 0, sizeof(unsigned int)* numBins));
  histo_kernel << <blocks, m >> >(d_cdf, d_logLuminance, numBins, logLumRange, min_logLum);

  //4) Perform an exclusive scan (prefix sum) on the histogram to get
  //   the cumulative distribution of luminance values (this should go in the
  //   incoming d_cdf pointer which already has been allocated for you)      
  //cdf_kernel << <1, numBins>> >(d_cdf, numBins);
  cdf_kernel_2 << <1, numBins, sizeof(unsigned int) * numBins * 2 >> >(d_cdf, numBins);
}